# Resolve Schema

**input**
```
+schema_path: str
```

**methods**
```
+read_json(schema_path: str): dict
+split_json(schema: dict): list
+resolver(entity1: dict, entity2: dict): dict
+resolve_defs(terms: dict, defs: dict) : dict
+ node_order(schema: dict): list
+resolve_nodes(nodeList: list, splitJsonList: list): list
+recombine_nodes(resolvedList: list) : dict
```

In [1]:
import logging
from datetime import datetime

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    filename=f"../logs/{datetime.now()}.log",
    filemode='x',
    force=True  # Ensures custom configuration takes effect
)

logging.info("This log should appear in logs.log")


In [2]:
import gen3_data_validator

## Reading in xlsx data and writing to json
- xlsx data comes from xlsx manifest file created from acdc_submission_template

In [3]:
# ResolverClass = gen3_data_validator.ResolveSchema(schema_path = "../schema/gen3_test_schema.json")
xlsxData = gen3_data_validator.ParseXlsxMetadata(xlsx_path = "../data/lipid_metadata_example.xlsx", skip_rows=1)
xlsxData.write_dict_to_json(output_dir="../data/restricted/lipid_metadata_example/")

JSON files written to ../data/restricted/lipid_metadata_example/


## Testing Linkage

In [4]:
Data = gen3_data_validator.ParseData(data_folder_path = "../data/restricted/lipid_metadata_example/")
Resolver = gen3_data_validator.ResolveSchema(schema_path = "../schema/gen3_test_schema.json")
Linkage = gen3_data_validator.TestLinkage(schema_resolver = Resolver, data_parser = Data)

=== Resolving Schema References ===
Resolved demographic.yaml
Resolved project.yaml
Resolved serum_marker_assay.yaml
Resolved alignment_workflow.yaml
Resolved imaging_file.yaml
Resolved lipidomics_assay.yaml
Resolved metabolomics_file.yaml
Resolved acknowledgement.yaml
Resolved medical_history.yaml
Resolved _settings.yaml
Resolved blood_pressure_test.yaml
Resolved genomics_assay.yaml
Resolved variant_file.yaml
Resolved program.yaml
Resolved serum_marker_file.yaml
Resolved proteomics_assay.yaml
Resolved sample.yaml
Resolved unaligned_reads_file.yaml
Resolved aligned_reads_index_file.yaml
Resolved variant_workflow.yaml
Resolved proteomics_file.yaml
Resolved exposure.yaml
Resolved metabolomics_assay.yaml
Resolved lipidomics_mapping_file.yaml
Resolved lipidomics_file.yaml
Resolved aligned_reads_file.yaml
Resolved lab_result.yaml
Resolved medication.yaml
Resolved publication.yaml
Resolved subject.yaml
Resolved core_metadata_collection.yaml
=== Validating Config Map ===
Root Node = ['subject

You can also bypass the data attribute in the linkage class and input your own data_map and config_map

In [ ]:
# Using the Linkage class which has the resolved schema with custom data and config
config_map = {
    "samples": {"primary_key": "sample_id", "foreign_key": "subject_id"},
    "files": {"primary_key": "file_id", "foreign_key": "sample_id"},
    "subjects": {"primary_key": "subject_id", "foreign_key": "project_id"},
    "project": {"primary_key": "project_id", "foreign_key": None}
}

data_map = {
    "samples": [
        {"sample_id": "sample_1", "subject_id": "subject_9"},
        {"sample_id": "sample_2", "subject_id": "subject_3"},  # Invalid FK
        {"sample_id": "sample_3", "subject_id": "subject_4"}, # Invalid FK
        {"sample_id": "sample_4", "subject_id": "subject_5"} # Invalid FK
    ],
    "files": [
        {"file_id": "file_1", "sample_id": "sample_1"},
        {"file_id": "file_2", "sample_id": "sample_27"}  # Invalid FK
    ],
    "subjects": [
        {"subject_id": "subject_1", "project_id": "project_1"},  
        {"subject_id": "subject_2", "project_id": "project_2"}, # Missing project 2
    ],
    "project": [
        {"project_id": "project_1"}
    ]
}

Linkage.validate_links(data_map, config_map)

# validation prototype


## Creating the validation class

In [ ]:
import gen3_data_validator

resolver = gen3_data_validator.ResolveSchema(schema_path = "../schema/gen3_test_schema.json")
data = gen3_data_validator.ParseData(data_folder_path = "../data/restricted/ausdiab_lipid_metadata/")
validator = gen3_data_validator.Validate(data_map=data.data_dict, resolved_schema=resolver.schema_resolved)

### Getting nested validation results
- returns a nested dictionary by entity/data node then by the row/index number, and then the validation objects

In [ ]:
validation_dict = validator.validation_result
validation_dict

In [ ]:
validator.list_entities()

In [ ]:
validator.list_index_by_entity("lipidomics_assay")

You can pull out a validation results for a specific entity with

In [ ]:
validator.pull_entity("lipidomics_assay")

You can pull validation results for a specific entity and then a specific index / row

In [ ]:
validator.pull_index_of_entity("lipidomics_assay", "index_1")

# Getting validation stats

In [ ]:
validate_stats = gen3_data_validator.ValidateStats(validator)
stats_df = validate_stats.summary_stats()
stats_df

# Creating validation summary data

In [9]:
Summary = gen3_data_validator.ValidateSummary(validator) 
flattened_results_dict = Summary.flatten_validation_results()
flattened_results_dict

### Converting flattened dict to pandas

In [ ]:
flatten_summary_pd = Summary.flattened_results_to_pd()
flatten_summary_pd

### Collapsing flattened dict to pandas
- This collapsed data frame summarises common validation errors

In [ ]:
collapse_df = Summary.collapse_flatten_results_to_pd()
collapse_df

# Writing validation results to folder

In [ ]:
import os
output_dir = "../data/restricted/ausdiab_lipid_metadata/validation/"
os.makedirs(output_dir, exist_ok=True)


def write_dict_to_json(input_dict, output_dir, filename:str):
    with open(f"{output_dir}/{filename}.json", "w") as f:
        json.dump(input_dict, f)
    print(f"JSON files written to {output_dir}")

write_dict_to_json(validation_dict, output_dir, "validation_dict")
write_dict_to_json(flattened_results_dict, output_dir, "flattened_results_dict")

# Writing pandas df
stats_df.to_csv(f"{output_dir}/stats_df.csv")
flatten_summary_pd.to_csv(f"{output_dir}/flatten_summary_pd.csv")
collapse_df.to_csv(f"{output_dir}/collapse_df.csv")


In [ ]:
# Use this for writing tests

sample_validation_results = {
    'sample': [
        [
            {
                'index': 0,
                'validation_result': 'FAIL',
                'invalid_key': 'freeze_thaw_cycles',
                'schema_path': 'properties.freeze_thaw_cycles.type',
                'validator': 'type',
                'validator_value': 'integer',
                'validation_error': "'10' is not of type 'integer'"
            },
            {
                'index': 0,
                'validation_result': 'FAIL',
                'invalid_key': 'sample_provider',
                'schema_path': 'properties.sample_provider.enum',
                'validator': 'enum',
                'validator_value': ['Baker', 'USYD', 'UMELB', 'UQ'],
                'validation_error': "45 is not one of ['Baker', 'USYD', 'UMELB', 'UQ']"
            },
            {
                'index': 0,
                'validation_result': 'FAIL',
                'invalid_key': 'sample_storage_method',
                'schema_path': 'properties.sample_storage_method.enum',
                'validator': 'enum',
                'validator_value': [
                    'not stored',
                    'ambient temperature',
                    'cut slide',
                    'fresh',
                    'frozen, -70C freezer',
                    'frozen, -150C freezer',
                    'frozen, liquid nitrogen',
                    'frozen, vapor phase',
                    'paraffin block',
                    'RNAlater, frozen',
                    'TRIzol, frozen'
                ],
                'validation_error': "'Autoclave' is not one of ['not stored', 'ambient temperature', 'cut slide', 'fresh', 'frozen, -70C freezer', 'frozen, -150C freezer', 'frozen, liquid nitrogen', 'frozen, vapor phase', 'paraffin block', 'RNAlater, frozen', 'TRIzol, frozen']"
            }
        ],
        [
            {
                'index': 1,
                'validation_result': 'FAIL',
                'invalid_key': 'freeze_thaw_cycles',
                'schema_path': 'properties.freeze_thaw_cycles.type',
                'validator': 'type',
                'validator_value': 'integer',
                'validation_error': "'76' is not of type 'integer'"
            },
            {
                'index': 1,
                'validation_result': 'FAIL',
                'invalid_key': 'sample_storage_method',
                'schema_path': 'properties.sample_storage_method.enum',
                'validator': 'enum',
                'validator_value': [
                    'not stored',
                    'ambient temperature',
                    'cut slide',
                    'fresh',
                    'frozen, -70C freezer',
                    'frozen, -150C freezer',
                    'frozen, liquid nitrogen',
                    'frozen, vapor phase',
                    'paraffin block',
                    'RNAlater, frozen',
                    'TRIzol, frozen'
                ],
                'validation_error': "'In the Pantry' is not one of ['not stored', 'ambient temperature', 'cut slide', 'fresh', 'frozen, -70C freezer', 'frozen, -150C freezer', 'frozen, liquid nitrogen', 'frozen, vapor phase', 'paraffin block', 'RNAlater, frozen', 'TRIzol, frozen']"
            }
        ],
        [
            {
                'index': 2,
                'validation_result': 'PASS',
                'invalid_key': None,
                'schema_path': None,
                'validator': None,
                'validator_value': None,
                'validation_error': None
            }
        ],
        [
            {
                'index': 3,
                'validation_result': 'PASS',
                'invalid_key': None,
                'schema_path': None,
                'validator': None,
                'validator_value': None,
                'validation_error': None
            }
        ]
    ]
}


